In [1]:
import numpy as np
import os
import time

In [3]:
# utf 8
import re

standardize_map = {
    'ê': 'e', 'è': 'e', 'é': 'e', 'ë': 'e', 'Ê' : 'E',
    'à': 'a', 'á': 'a', 'â': 'a', 'ä' : 'a', 'ă' :'a',
    'î': 'i', 'ì': 'i', 'í': 'i', 'ï': 'i',
    'ô': 'o', 'ò': 'o', 'ó': 'o', 'ö': 'o',
    'ù': 'u', 'ú': 'u', 'û': 'u', 'ü': 'u'
}

def standardize_text(text):
    for diacritic_char, standard_char in standardize_map.items():
        text = text.replace(diacritic_char, standard_char)
    return text

with open('scrapped.txt', 'r', encoding='utf-8') as file:
    text = file.read()
    file.close()

standardized_text = standardize_text(text).lower()

with open('standardized.txt', 'w', encoding='utf-8') as file:
    file.write(standardized_text)
    file.close()



In [4]:
# remove punc
import re
import string

def remove_punctuation(text):

    pattern = f"[{re.escape(string.punctuation)}]"
    text_no_punctuation = re.sub(pattern, "", text)
    
    return text_no_punctuation

v2_standardized_text = remove_punctuation(standardized_text)

with open('standardized-2.txt', 'w', encoding='utf-8') as file:
    file.write(v2_standardized_text)
    file.close()


In [5]:
# remove empty lines
def remove_empty_lines(text):
    lines = text.split("\n")
    
    non_empty_lines = [line for line in lines if line.strip()]
    
    cleaned_text = "\n".join(non_empty_lines)
    
    return cleaned_text    


v3_standardized_text = remove_empty_lines(v2_standardized_text)

with open('standardized-3.txt', 'w', encoding='utf-8') as file:
    file.write(v3_standardized_text)
    file.close()

In [6]:
# count
def count_words(text):
    words = text.split()
    return len(words)

word_count = count_words(v2_standardized_text)

print("total :", word_count)


total : 9365


In [7]:
# unique
def get_unique_words(text):
    words = text.split()
    
    unique_words = set(word.lower() for word in words)
    
    return list(unique_words)

unique_words = get_unique_words(v3_standardized_text)
length_of_unique_words = len(unique_words)

print("Jumlah Kata Unik:", len(unique_words))


Jumlah Kata Unik: 2885


In [8]:
# [[],[]]
def text_to_2d_list(text):

    lines = text.split("\n")
    
    words_2d_list = [line.split() for line in lines if line.strip()]
    
    return words_2d_list


words_2d_list = text_to_2d_list(v3_standardized_text)

print(words_2d_list[:2])
# v3_standardized_text


[['lingkup', 'pencarian', 'teks', 'dan', 'catatankakinya', 'teks', 'pencarian', '224', 'karakter', 'filter', 'pencarian', 'huruf', 'besarkecil', 'diakritik', 'serta', 'pungtuasi', 'diabaikan', 'karakter', 'dapat', 'digunakan', 'sebagai', 'pengganti', 'zero', 'atau', 'satu', 'huruf', 'sembarang', 'simbol', 'wildcard', 'dapat', 'digunakan', 'sebagai', 'pengganti', 'zero', 'atau', 'sejumlah', 'karakter', 'termasuk', 'spasi', 'mengakomodasi', 'variasi', 'ejaan', 'antara', 'lain', 'dj', 'j', 'tj', 'c', 'j', 'y', 'oe', 'u', 'd', 'dh', 't', 'th'], ['anggitanipun', 'dawud', 'magang', 'guru', 'ing', 'masaran']]


In [9]:
def build_indices(unique_words):
    word_to_idx = {}
    idx_to_word = {}
    for i, word in enumerate(unique_words):
        word_to_idx[word] = i
        idx_to_word[i] = word
    return word_to_idx, idx_to_word

word_to_idx, idx_to_word = build_indices(unique_words)

In [10]:
# word_to_idx

for i, (word, idx) in enumerate(word_to_idx.items()):
    print(f"{word}: {idx}")
    if i == 9:
        break


anginggahi: 0
grayang: 1
ngeningaken: 2
godha: 3
jaler: 4
lengenku: 5
ngampet: 6
pa: 7
kasimpena: 8
milar: 9


In [11]:
def prepare_corpus(corpus, word_to_idx):
    sequences = []
    
    for line in corpus:
        tokens = line
        missing_tokens = [token for token in tokens if token not in word_to_idx]
        
        for token in missing_tokens:
            word_to_idx[token] = len(word_to_idx)  
            print(f"Token '{token}' ditambahkan ke word_to_idx.")
        
        for i in range(1, len(tokens)):
            i_gram_sequence = tokens[:i+1]
            i_gram_sequence_ids = [
                word_to_idx[token] for token in i_gram_sequence
            ]
            sequences.append(i_gram_sequence_ids)
    
    return sequences

sequences = prepare_corpus(words_2d_list, word_to_idx)
max_sequence_len = max([len(x) for x in sequences])

In [12]:
sequences[0]

[2394, 899]

In [14]:
print(idx_to_word[1647])
print(idx_to_word[867])
print(idx_to_word[1452])

cumeplong
milanipun
katamakake


In [15]:
len(sequences)

9210

In [16]:
max_sequence_len

146

In [17]:
from keras.preprocessing.sequence import pad_sequences
from keras import utils
import numpy as np

def build_input_data(sequences, max_sequence_len, length_of_unique_words):
    sequences = np.array(pad_sequences(sequences, maxlen = max_sequence_len, padding = 'pre'))
    X = sequences[:,:-1]
    y = sequences[:,-1]
    y = utils.to_categorical(y, length_of_unique_words)
    return X, y

X, y = build_input_data(sequences, max_sequence_len, length_of_unique_words)


2024-12-04 16:03:58.518158: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-04 16:04:00.959563: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-04 16:04:02.312595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733303043.862123   59553 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733303044.060097   59553 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 16:04:05.671664: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [21]:
from keras.layers import Dense, LSTM, Dropout, Embedding, BatchNormalization, Bidirectional
from keras.models import Sequential

def create_model(max_sequence_len, length_of_unique_words):
    model = Sequential([
        Embedding(length_of_unique_words, 64, input_length= max_sequence_len-1),
        Bidirectional(LSTM(64,return_sequences=True)),
        Bidirectional(LSTM(64)),
        BatchNormalization(),
        Dense(128, activation="relu"),
        Dense(length_of_unique_words, activation="softmax")
    ])
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'] 
    )
    
    return model


In [23]:
model = create_model(max_sequence_len, length_of_unique_words)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
len(X)

9210

In [27]:
model.fit(X, y, batch_size = 512, epochs=30)

Epoch 1/30


2024-12-04 16:17:01.474013: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 106283400 exceeds 10% of free system memory.


18/18 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.0361 - loss: 6.9255
Epoch 2/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.0341 - loss: 6.7826
Epoch 3/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.0335 - loss: 6.6794
Epoch 4/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.0337 - loss: 6.5740
Epoch 5/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.0357 - loss: 6.4784
Epoch 6/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.0342 - loss: 6.3969
Epoch 7/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.0324 - loss: 6.2890
Epoch 8/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.0337 - loss: 6.1214
Epoch 9/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.0399 - loss: 5.9812
Epoch 10/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.0471 - loss: 5.8277
Epoch 11/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.0549 - loss: 5.6386
Epoch 12/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.0619 - loss: 5.5045


In [28]:
# def generate_text(seed_text, next_words, model, max_seq_len):
#     for _ in range(next_words):
#         sequences= prepare_corpus(words_2d_list[2], word_to_idx)
#         sequences = pad_sequences([sequences[-1]], maxlen=max_seq_len-1, padding='pre')
#         predicted = model.predict_classes(sequences, verbose=0)
#         output_word = ''
#         output_word = idx_to_word[predicted[0]]            
#         seed_text = seed_text + " " + output_word
        
#     return seed_text.title()

from keras.preprocessing.sequence import pad_sequences

def generate_text(seed_text, next_words, model, max_seq_len, word_to_idx, idx_to_word):
    for _ in range(next_words):
        tokens = seed_text.split()
        token_sequence = [word_to_idx.get(token, 0) for token in tokens] 
        
        padded_sequence = pad_sequences([token_sequence], maxlen=max_seq_len - 1, padding='pre')
        
        predicted_index = np.argmax(model.predict(padded_sequence, verbose=0))
        predicted_word = idx_to_word[predicted_index]
        
        seed_text += f" {predicted_word}"
    
    return seed_text


In [29]:
print(generate_text("kacariyos ing jaman", 30, model, max_sequence_len, word_to_idx, idx_to_word))

kacariyos ing jaman kina satunggiling wektu sang nata angandika dhateng anom ana ing pundhut putra putra priyantun jagi kawula abdi kawula kalilana andharaken ingkang prakawis ingkang badhe prakawis makaten punika minangka pratandha tementemening


In [50]:
model_structure = model.to_json()
with open("v1_text_generation.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("v1_text_generation.weights.h5")